In [ ]:

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model


In [ ]:

#load trained model
model = load_model('age_gender_model.h5', compile=False)
print("Model loaded successfully")


In [ ]:

def predict_age_gender(image_path, model, image_size=128):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (image_size, image_size)) / 255.0
    img_exp = np.expand_dims(img, axis=0)

    age_pred, gender_pred = model.predict(img_exp)
    age = int(age_pred[0][0])
    gender = "Female" if np.argmax(gender_pred[0]) == 1 else "Male"

    return age, gender, img


In [ ]:

#predict from local image file
from tkinter import Tk, filedialog

def upload_and_predict_local():
    root = Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select an image",
        filetypes=[("Image files", "*.jpg *.jpeg *.png")]
    )

    if file_path:
        age, gender, img = predict_age_gender(file_path, model)
        plt.imshow(img)
        plt.title(f"Predicted Age: {age}, Gender: {gender}")
        plt.axis('off')
        plt.show()

        print(f"Predicted Age: {age}")
        print(f"Predicted Gender: {gender}")
    else:
        print("No file selected.")


In [ ]:

#predict from webcam
def capture_and_predict_webcam():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Webcam access failed.")
        return

    print("Press 's' to capture and predict, 'q' to quit.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame.")
            break

        cv2.imshow("Webcam - Press 's' to capture", frame)
        key = cv2.waitKey(1) & 0xFF

        if key == ord('s'):
            path = "captured_image.jpg"
            cv2.imwrite(path, frame)
            age, gender, img = predict_age_gender(path, model)

            plt.imshow(img)
            plt.title(f"Predicted Age: {age}, Gender: {gender}")
            plt.axis('off')
            plt.show()

        elif key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [ ]:
print("1. Predict from webcam")
print("2. Upload image from file")

choice = input("Enter 1 or 2: ")

if choice == '1':
    capture_and_predict_webcam()
elif choice == '2':
    upload_and_predict_local()
else:
    print("Invalid choice.")
